# Polynomial Functions

In [10]:
import numpy as np
import itertools
import scipy



#Easier to read polynomial function
def poly(lst):  
    return np.poly1d(lst)

#To extract the degree of the polynomial, use:
def deg(poly):  
    return poly.order

#To extract the leading coefficient of the polynomial:
def leadingcoff(poly):
    return int(poly.c[0])

#Takes derivative
def deriv(f):
    return np.polyder(f)

#x^k generator
#INPUT: k
#OUTPUT: x^k
def monomial_k(k):
    lst = []
    for i in range(0, k+1):  
        if i != 0:
            lst.append(0)    
        else:
            lst.append(1)        
    return poly(lst)

#Takes polynomials from Z[x] into F_p[x]
#INPUT: polynomial in Z[x]
#OUTPUT: polynomial in F_p[x]
def ZtoFq(poly, p):
    
    new_coeff = []
    d = deg(poly)
    
    for k in range(0, d + 1):
        
        new_coeff.append((poly[d - k] % p))  
        
    return np.poly1d(new_coeff)

#I'm stupid. I'm so stupid
#INPUT: f, g polynomials in Z/pZ[x]
#OUTPUT: the image of f under the quotient map into Z/pZ[x]/(g)
def fasterquotientmap(f, g, p):
    
    return polydiv(f, g, p)[1]

#Generates list of all possible lists of polynomial coefficients in Z/pZ with degree strictly less than d
def lstofpolynomials(p, d):
    
    lst = []
    
    for coeffs in itertools.product(range(p), repeat = d):
        lst.append(coeffs)

    return lst

def lstofmonicpolynomials(p, d):
    
    lst = []
    
    for coeffs in itertools.product(range(p), repeat = d):
        lst.append([1] + list(coeffs))
        
    return lst

#Performs polynomial long division
def polydiv(f, g, p):
    f = ZtoFq(f, p)
    g = ZtoFq(g, p)

    if g == poly([0]):
        return "I am so dissappointed"

    q = poly([0])
    r = f

    while (r != poly([0])) and (deg(r) >= deg(g)):
        # This is what we subtract by during long division
        coeff = (leadingcoff(r) * inverse(int(leadingcoff(g)), p)) % p
        t = coeff * monomial_k(deg(r) - deg(g))

        q_new = q + t
        r_new = r - t * g

        q = ZtoFq(q_new, p)
        r = ZtoFq(r_new, p)

    return (q, r)

#INPUT: a polynomial in f in Z/pZ[x]
#OUTPUT: True if f is squarefree in Z/pZ[x], False else
def squarefree(f, p):
    
    g = ZtoFq(deriv(f), p)
    
    h = gcdfinder(f, g, p)[2]
    
    if deg(h) == 0:
        return True
    
    else:
        return False


# Finite Field Tools

In [2]:
import math

# Applies the Euclidean Algorithm to compute the gcd of polynomials f and g in Z/pZ[x]
# The output array (u0, v0, r0) are the polynomials for f and g such that
#            u0*f + v0*g = r0
# If the inputs are coprime, then r0 will be a constant
def gcdfinder(f, g, p):
    r0 = f
    r1 = g
    u0 = poly([1])
    v0 = poly([0])
    u1 = poly([0])
    v1 = poly([1])

    while (r1 != poly([0])):
        quot = polydiv(r0, r1, p)[0]

        R = ZtoFq(-quot * r1 + r0, p)
        r0 = r1
        r1 = R

        U = ZtoFq(-quot * u1 + u0, p)
        u0 = u1
        u1 = U

        V = ZtoFq(-quot * v1 + v0, p)
        v0 = v1
        v1 = V

    return (u0, v0, r0)

#Computes the inverse of x in Z/pZ
def inverse(x, p):
    inv = 1
    for k in range(1, p):
        if (k*x % p == 1):
            inv = k
    return inv

#INPUT: polynomial g in Z/pZ[x]
#OUTPUT: a monic polynomial g' such that (g) = (g') in Z/pZ[x]
def makemonic(g, p):
    f = inverse(leadingcoff(g), p)*g
    return ZtoFq(f, p)

#Brute force irreducibility checker for polynomials in Z/pZ[x]
def irreducible(f, p):
    
    d = deg(f)
        
    for poly in lstofpolynomials(p, d):
        
        if deg(np.poly1d(poly)) == 0:
            
            continue
        
        if polydiv(f, np.poly1d(poly), p)[1] == np.poly1d([0]):
            
            return False
    
    return True

# Random Generator Tools

In [3]:
import random as rnd


#Generates a random polynomial of degree d in Z/pZ[x]
def randompoly(d, p):
    
    lst = [rnd.randint(1, p-1)]
    
    for l in range(1, d+1):
        
        lst.append(rnd.randint(0, p-1))
    
    return np.poly1d(lst)

#Generates a random irreducible polynomial of degree d in Z/pZ[x]
def randomirreducible(d, p):
    
    f = randompoly(d, p)
    
    while irreducible(f, p) == False:
        
        f = randompoly(d, p)
    
    return f

#Generates a random squarefree polynomial of degree d in Z/pZ[x]

def randomsquarefree(d, p):
    
    f = randompoly(d, p)
    
    while squarefree(f, p) == False:
        
        f = randompoly(d, p)
        
    return f


# Legendre Symbol

In [4]:
from sympy.ntheory import legendre_symbol

def legendresymbol(f, h, p):
    
    legsym = 1
    h = makemonic(h, p)
    
    while True:
        
        if (deg(h) == 0) and (h != poly([0])):
            return 1
        
        f = fasterquotientmap(f, h, p)
        
        if (f == poly([0])):
            return 0
        
        c = leadingcoff(f)
        
        if (deg(f) == 0):    
            return (legendre_symbol(int(c), p)**(deg(h)))*legsym
        
        f = ZtoFq(inverse(c, p)*f, p)
        
        if (((p-1)/2) % 2 == 1) and (deg(f) % 2 == 1) and (deg(h) % 2 == 1):
            
            legsym = -legendre_symbol(int(c), p)**(deg(h))*legsym
            
        else:
            
            legsym = legendre_symbol(int(c), p)**(deg(h))*legsym
            
        G = h
        h = f
        f = G
   

In [6]:
deg(poly([0]))

0

# Brute Force Legendre Symbol

In [5]:
#Computes the set of squares in the quotient ring F_p[T]/(g) where deg(g) = d (Only works for q = p prime and odd)
#This code also requires that g is monic or else the image under the quotient map will get... weird
#For convenience, g will be become a monic polynomial that generates the same ideal as g
def lstofsquares(g, p):
    
    g = makemonic(g, p)
    
    d = deg(g)
    
    lst = lstofpolynomials(p, d)
    
    sqlst = []
    
    for poly in lst:
        
        f = fasterquotientmap(np.poly1d(poly)*np.poly1d(poly), g, p)

        sqlst.append(list(f.c))
    
    #This line of code erases duplicate entries in the list of squares
    sqlst = [k for k,v in itertools.groupby(sorted(sqlst))]
        
    return sqlst

#WARNING: This method only works for irreducible g.
#Remember that the Legendre symbol defined as f = square mod g works for g irreducible.
#When g is not, we are working with the Jacobi symbol which does not have the same residue meaning as the
#Legendre symbol! i.e. (f/g) = 1 does not imply that f = square mod g
def bruteforcelegendrecomputation(f, g, p):
    
    g = makemonic(g, p)
    f = fasterquotientmap(f, g, p)
    f_coeff = list(f.c)
    
    if f_coeff == [0]:
        
        return 0
    
    if (f_coeff in lstofsquares(g, p)) == True:
        
        return 1
    
    if (f_coeff in lstofsquares(g, p)) == False:
        
        return -1
        

# Legendre Symbol Verifier

In [6]:
#Performs n random tests of the Legendre symbol between two polynomials of at most degree d
#Compares outputs of Legendresymbol function and bruteforcelegendrecomputation function
def testlegendresymbol(n, d):
    
    k = 0
    primes = [3, 5, 7, 11, 13, 17, 19, 23]
    
    while k < n:
        
        p = primes[rnd.randint(0, 7)]
        
        deg1 = rnd.randint(0, d)
        deg2 = rnd.randint(1, d)
        
        f = randompoly(deg1, p)
        print(f)
        
        g = randomirreducible(deg2, p)
        print(g)
        print(irreducible(g, p))
        
        print(p)
        print(legendresymbol(f, g, p))
        print(bruteforcelegendrecomputation(f, g, p))
        print(legendresymbol(f, g, p) == bruteforcelegendrecomputation(f, g, p))
            
        k = k + 1
    

# L-function

Let $\mathbb{F}_q$ be a finite field with characteristic $p$ and let $D(x) \in \mathbb{F}_q[x]$ be a monic, squarefree polynomial of degree $d.$ Let $R$ be the ring defined as

$$R : = \left\{ a(x) + b(x) \sqrt{D(x)} \ : \ a(x), b(x) \in \mathbb{F}_q[x] \right\}.$$

Let $P(x)$ be an irreducible polynomial in $\mathbb{F}_q[x]$ and $a(x) \in \mathbb{F}_q[x]$. Then we define the *Legendre Symbol* as

$$
\left( \frac{a(x)}{P(x)} \right) : =
\begin{cases}
1, & \text{if } P(x) \nmid a(x) \text{ and } a(x) \text{ is a square modulo } P(x), \\
-1, & \text{if } P(x) \nmid a(x) \text{ and } a(x) \text{ is not a square modulo } P(x), \\
0, & \text{if } P(x) \mid a(x).
\end{cases}
$$

We also have an analog to the *Jacobi Symbol* with

$$\left( \frac{a(x)}{b(x)} \right) = \prod_{j = 1}^r \left( \frac{a(x)}{Q_j(x)} \right)^{\alpha_j}$$

where $a(x), b(x) \in \mathbb{F}_q[x]$ and $b(x) = Q_1(x)^{\alpha_1} \cdots Q_r(x)^{\alpha_r}.$

Furthermore, we have the quadratic reciprocity law for $a(x), b(x) \in \mathbb{F}_q[x]$, relatively prime, non-zero and monic

$$\left(\frac{a(x)}{b(x)} \right) \left( \frac{b(x)}{a(x)} \right) = (-1)^{\frac{|a| - 1}{2} \frac{|b| - 1}{2}} = (-1)^{\frac{q-1}{2} \text{deg} a(x) \cdot \text{deg} b(x)}$$

where $|f(x)| = q^{\text{deg}f}.$

For our sake, for polynomials $D(x), n(x) \in \mathbb{F}_q[x]$ we define the character
$$\chi_D (n) : = \left( \frac{D(x)}{n(x)} \right).$$
Additionally, as per usual, we define the *L-function* of $\chi_D$ as
$$L(s, \chi_D) : = \sum_{\substack{n \in \mathbb{F}_q[x], \text{ monic} \\ \text{deg}(n) \geq 0}} \frac{\chi_D(n)}{|n|^s}$$
where $|n| = q^{\text{deg} n}.$ By the definition of $|\cdot|$, we can rearrange the sum to get
$$L(s, \chi_D) = \sum_{r = 0}^{\infty} q^{-rs} \sum_{\substack{n \in \mathbb{F}_q[x], \text{ monic} \\ \text{deg }n = r}} \chi_D(n).$$

In [7]:
# the sum of the legendre symbol of polynomial D(x) of all monic polynomials in Z/pZ[x] of degree exactly r

def innersum(D, r, p):
    
    summm = 0
    
    for item in lstofmonicpolynomials(p, r):
        
        summm = summm + legendresymbol(D, poly(item), p)
        
    return summm

# The L-function with character \chi_D
# This is so bad. It is way too slow
def Lfunction(s, D, p):
    
    summ = 0
    
    D = makemonic(D, p)
    
    for r in range(0, deg(D)):
        summ = summ + (p**(-r*s))*innersum(D, r, p)
    
    return summ

#This will output the Lfunction for D as a polynomial
def polyLfunction(D, p):
    D = makemonic(D, p)
    d = deg(D)
    lst = []
    
    for r in range(0, d):
        
        lst.append(innersum(D, d - 1 - r, p))
        
    return poly(lst)
        

In [8]:
summ = 0
for k in range(0, 4):
    test = innersum(poly([1, 0, 2, 1, 3]), k, 5)
    print("Innersum:", k, test)
    summ = summ + test
    print(summ)

Innersum: 0 1
1
Innersum: 1 -1
0
Innersum: 2 5
5
Innersum: 3 -5
0


In [25]:
Lfunction(0,poly([1, 0, 2, 1, 3]), 5)

0

In [15]:
innersum(poly([1, 0, 2, 1, 3]), 5, 5)

0

In [19]:
legendresymbol(poly([1, 0, 2, 1, 3]), poly([1]), 5)

0

In [18]:
f = randomsquarefree(6, 5)
print(f)

   6     5     4     3     2
1 x + 2 x + 3 x + 1 x + 4 x + 4 x


In [19]:
f

poly1d([1, 2, 3, 1, 4, 4, 0])

In [23]:
g

poly1d([-25,  20,   6,  -2,   0,   1])

In [24]:
np.polydiv(g, poly([-1, 1]))

(poly1d([25.,  5., -1.,  1.,  1.]), poly1d([0.]))

In [25]:
makemonic(poly([3, 3, 0, 4, 2, 2, 2]), 5)

poly1d([1, 1, 0, 3, 4, 4, 4])

# Using the Z_C

For a curve $C$ we can associate a zeta-function defined as

$$Z_C(u) : = \exp \left( \sum_{r =1}^{\infty} N_r(C) \frac{u^r}{r} \right).$$

Recall the usual moment problem of computing the sum

$$ \lim_{q \to \infty} \frac{1}{\# \mathcal{H}_{q, d}} \sum_{D(x) \in \mathcal{H}_{q, d}} L(1/2, \chi_D)^k.$$



In [24]:
def binomial(k, l):
    
    return (math.factorial(k)/(math.factorial(k - l)*math.factorial(l)))

def c(k, l, q):
    
    if l > k:
        return 0
    
    solution = ((1 + q**2)**(k-l))*(q**(l/2))
    solution = solution*binomial(k,l)
    solution = solution*((-1)**l)
    
    if int(solution) == solution:
        
        return int(solution)
    
    else:
        
        return solution

def fqkmaker(k, q):
    
    lst = []
    
    for l in range(0, k + 1):
    
        lst.append(c(k, k-l, q))
        
    return poly(lst)

In [43]:
fqkmaker(3, 9)

poly1d([   -27,   2214, -60516, 551368])

25.0